<a class="anchor" id="0."></a>
# **Contents**


1. [FMNIST Image Classification](#1.)
* * [1.1. Load FMNIST Dataset](#1.1.)
* * [1.2. Prepare Dataloaders](#1.2.)
* * [1.3. Create Model](#1.3.)
* * [1.4. Define Loss Function and Optimizer](#1.4.)
* * [1.5. Train Model](#1.5.)
* * [1.6. Save Model](#1.6.)
* * [1.7. Load Model](#1.7.)

In [33]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



[Go to Contents Menu](#0.)
<a class="anchor" id="1."></a>

# **1. FMNIST Image Classification**  

device: cuda


[Go to Contents Menu](#0.)
<a class="anchor" id="1.1."></a>

# **1.1. Load FMNIST Dataset**  

In [7]:
train_data = datasets.FashionMNIST(
root='data',
train=True,
download=True,
transform=ToTensor()
)

test_data = datasets.FashionMNIST(
root='data',
train=False,
download=True,
transform=ToTensor()
)

[Go to Contents Menu](#0.)
<a class="anchor" id="1.2."></a>

# **1.2. Prepare Dataloaders**  

In [11]:
BATCH_SIZE = 64
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of S [N, C, H, W]: {X.shape}")
    break

Shape of S [N, C, H, W]: torch.Size([64, 1, 28, 28])


[Go to Contents Menu](#0.)
<a class="anchor" id="1.3."></a>

# **1.3. Create Model**  

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [27]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN().to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


[Go to Contents Menu](#0.)
<a class="anchor" id="1.4."></a>

# **1.4. Define Loss Functions and Optimizer**  

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

[Go to Contents Menu](#0.)
<a class="anchor" id="1.5."></a>

# **1.5. Train the Model**  

In [29]:
def train(model, dataloader, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_no, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_no % 100 == 0:
            loss, current = loss.item(), batch_no * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")



In [30]:
def test(model, dataloader, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    acc = correct / size
    print(f"Acc:{100*acc:>0.1f}% Avg loss:{test_loss:>8f}")

In [31]:
epochs = 5
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}\n--------------------------------------")
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, test_dataloader, loss_fn)
print("Done...")

Epoch 1
--------------------------------------
loss:2.302145 [    0/60000]
loss:2.285991 [ 6400/60000]
loss:2.269955 [12800/60000]
loss:2.267660 [19200/60000]
loss:2.244392 [25600/60000]
loss:2.214330 [32000/60000]
loss:2.226847 [38400/60000]
loss:2.189646 [44800/60000]
loss:2.181600 [51200/60000]
loss:2.158941 [57600/60000]
Acc:35.9% Avg loss:2.153218
Epoch 2
--------------------------------------
loss:2.158802 [    0/60000]
loss:2.145091 [ 6400/60000]
loss:2.096629 [12800/60000]
loss:2.119261 [19200/60000]
loss:2.058169 [25600/60000]
loss:1.995608 [32000/60000]
loss:2.026676 [38400/60000]
loss:1.945213 [44800/60000]
loss:1.945575 [51200/60000]
loss:1.883203 [57600/60000]
Acc:55.0% Avg loss:1.881862
Epoch 3
--------------------------------------
loss:1.910474 [    0/60000]
loss:1.876029 [ 6400/60000]
loss:1.771181 [12800/60000]
loss:1.816827 [19200/60000]
loss:1.697879 [25600/60000]
loss:1.650708 [32000/60000]
loss:1.676609 [38400/60000]
loss:1.581342 [44800/60000]
loss:1.600827 [5120

[Go to Contents Menu](#0.)
<a class="anchor" id="1.6."></a>

# **1.6. Save Model**  

In [34]:
save_folder = "/home/serkan/repos_personel"
model_name = "fmnist_model.pth"
model_path = os.path.join(save_folder, model_name)
torch.save(model.state_dict(), model_path)
print("Saved Pytorch Model State to {}".format(model_name))


Saved Pytorch Model State to fmnist_model.pth


[Go to Contents Menu](#0.)
<a class="anchor" id="1.7."></a>

# **1.7. Load Model**  

In [35]:

loaded_model = NN()
loaded_model.load_state_dict(torch.load(model_path))
print("Loaded model:", model_path)

Loaded model: /home/serkan/repos_personel/fmnist_model.pth


[Go to Contents Menu](#0.)
<a class="anchor" id="1.8."></a>

# **1.8. Make Prediction**  

In [40]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

loaded_model.eval()
n = 100
x, y = test_data[n][0], test_data[n][1]
with torch.no_grad():
    pred = loaded_model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted:{predicted}, Actual:{actual}")

Predicted:Dress, Actual:Dress
